## Langchain Expression Language Basics

-  LangChain Expression Language is that any two runnables can be "chained" together into sequences. 
- The output of the previous runnable's .invoke() call is passed as input to the next runnable.
- This can be done using the pipe operator (|), or the more explicit .pipe() method, which does the same thing.

- Type of LCEL Chains
    - SequentialChain
    - Parallel Chain
    - Router Chain
    - Chain Runnables
    - Custom Chain (Runnable Sequence)

In [1]:
from dotenv import load_dotenv

load_dotenv('./../../.env')

True

### Sequential LCEL Chain

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        ChatPromptTemplate
                                        )

base_url = "http://localhost:11434"
model = 'llama3.2:3b'

llm = ChatOllama(base_url=base_url, model=model)
llm

ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')

In [3]:
system = SystemMessagePromptTemplate.from_template('Eres un profesor de {school}. Genera respuestas cortas.')

question = HumanMessagePromptTemplate.from_template('dime acerca de {topics} en {points} puntos')


messages = [system, question]
template = ChatPromptTemplate(messages)

question = template.invoke({'school': 'primaria', 'topics': 'sistema solar', 'points': 5})

response = llm.invoke(question)
print(response.content)    

Claro, aquí te presento 5 puntos clave sobre el Sistema Solar:

1. **Definición**: El Sistema Solar es la colección de planetas, lunas, asteroides y cometas que orbitan alrededor del Sol.

2. **Componentes principales**: 
   - El Sol (núcleo) 
   - 8 planetas (Mercurio, Venus, Tierra, Marte, Júpiter, Saturno, Urano y Neptuno)
   - Lunas de estos planetas

3. **Orbita**: Cada planeta sigue una órbita alrededor del Sol debido a la fuerza gravitatoria.

4. **Estructura**: El Sistema Solar se divide en 2 zonas principales: 
   - La zona interna (Mercurio y Venus)
   - La zona externa (Júpiter, Saturno, Urano y Neptuno)

5. **Importancia**: Es fundamental para entender el lugar de nuestro planeta, la Tierra, en el universo y nuestra posición en el Sistema Solar.


In [4]:
chain = template | llm

In [5]:
response = chain.invoke({'school': 'primaria', 'topics': 'sistema solar', 'points': 5})
print(response.content)

Aquí te presento 5 puntos clave sobre el Sistema Solar:

1. **Definición**: El Sistema Solar es la colectividad de planetas, lunas y demás objetos que orbitan alrededor del Sol.

2. **Orbita del Sol**: El Sol se encuentra en el centro del Sistema Solar y es la fuente de luz y calor para todos los planetas y objetos que lo rodean.

3. **Planetas principales**: Los 8 planetas más grandes del Sistema Solar, en orden desde la más cercana al Sol, son Mercurio, Venus, Tierra, Marte, Júpiter, Saturno, Urano y Neptuno.

4. **Satélites**: Cada planeta tiene sus propios satélites, como Luna (la luna de la Tierra), que orbitan alrededor de ellos.

5. **Objetos restantes**: El Sistema Solar también incluye asteroides, cometas y otros objetos menores que orbitan el Sol o los planetas.


In [6]:
response = chain.invoke({'school': 'posgrado', 'topics': 'sistema solar', 'points': 5})
print(response.content)

Aquí te presento los 5 puntos clave sobre el Sistema Solar:

1. **Definición**: El Sistema Solar es la colección de planetas, lunas, asteroides, cometas y otros objetos que orbitan alrededor del Sol.

2. **Origen**: Se cree que el Sistema Solar se formó hace aproximadamente 4.600 millones de años a partir de una nube de gas y polvo.

3. **Componentes principales**: El Sistema Solar comprende: el Sol (el centro), los planetas Mercurio, Venus, Tierra, Marte, Júpiter, Saturno, Urano y Neptuno, así como sus respectivas lunas.

4. **Escala**: La distancia entre el Sol y cada planeta varía desde aproximadamente 58 millones de kilómetros (Mercurio) hasta 1,4 billones de kilómetros (Neptuno).

5. **Evolución**: El Sistema Solar ha cambiado con el tiempo debido a la accionar de fuerzas como la gravedad y las colisiones entre objetos en orbita.


In [7]:
response

AIMessage(content='Aquí te presento los 5 puntos clave sobre el Sistema Solar:\n\n1. **Definición**: El Sistema Solar es la colección de planetas, lunas, asteroides, cometas y otros objetos que orbitan alrededor del Sol.\n\n2. **Origen**: Se cree que el Sistema Solar se formó hace aproximadamente 4.600 millones de años a partir de una nube de gas y polvo.\n\n3. **Componentes principales**: El Sistema Solar comprende: el Sol (el centro), los planetas Mercurio, Venus, Tierra, Marte, Júpiter, Saturno, Urano y Neptuno, así como sus respectivas lunas.\n\n4. **Escala**: La distancia entre el Sol y cada planeta varía desde aproximadamente 58 millones de kilómetros (Mercurio) hasta 1,4 billones de kilómetros (Neptuno).\n\n5. **Evolución**: El Sistema Solar ha cambiado con el tiempo debido a la accionar de fuerzas como la gravedad y las colisiones entre objetos en orbita.', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-01-13T22:25:37.669251Z', 'message': {

In [9]:
from langchain_core.output_parsers import StrOutputParser

In [10]:
chain = template | llm | StrOutputParser()
response = chain.invoke({'school': 'posgrado', 'topics': 'sistema solar', 'points': 5})
print(response)

Claro, aquí te presento 5 puntos clave sobre el Sistema Solar:

1. **Definición**: El Sistema Solar es la colección de planetas, lunas, asteroides, cometas y otras regiones del espacio que orbitan alrededor del Sol.

2. **Componentes**: Los componentes principales son: 
   - Mercurio
   - Venus
   - Tierra
   - Marte
   - Júpiter
   - Saturno
   - Urano
   - Neptuno

3. **Estructura**: El Sistema Solar se puede dividir en 3 regiones principales:
   - **Zona interna**: Mercurio, Venus y Tierra.
   - **Zona del Sol**: La Tierra y los asteroides.
   - **Zona externa**: Los planetas gasosos (Júpiter, Saturno) y Neptuno.

4. **Formación**: Se cree que el Sistema Solar se formó hace unos 4,600 millones de años a partir de una nube de gas y polvo que colapsó bajo su propia gravedad.

5. **Evolución**: El Sistema Solar sigue evolucionando debido a la interacción gravitatoria entre sus componentes y los impactos externos como meteoritos y otros asteroideos.


### Chaining Runnables (Chain Multiple Runnables)

- We can even combine this chain with more runnables to create another chain.
- Let's see how easy our generated output is?

In [11]:
chain

ChatPromptTemplate(input_variables=['points', 'school', 'topics'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='Eres un profesor de {school}. Genera respuestas cortas.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['points', 'topics'], input_types={}, partial_variables={}, template='dime acerca de {topics} en {points} puntos'), additional_kwargs={})])
| ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')
| StrOutputParser()

In [12]:
analysis_prompt = ChatPromptTemplate.from_template('''Analiza el siguiente texto: {response}
                                                   Dime que tan dificil es entenderlo.
                                                   Responde en una sola oracion.
                                                   ''')

fact_check_chain = analysis_prompt | llm | StrOutputParser()
output = fact_check_chain.invoke({'response': response})
print(output)

Entender el Sistema Solar puede ser un desafío, especialmente para aquellos que no tienen una base de conocimiento científico previa, pero con información y recursos adecuados, es posible lograr una comprensión clara.


In [13]:
composed_chain = {"response": chain} | analysis_prompt | llm | StrOutputParser()

output = composed_chain.invoke({'school': 'basica primaria', 'topics': 'sistema solar', 'points': 5})
print(output)

Entender los puntos clave sobre el Sistema Solar no es particularmente difícil.


### Parallel LCEL Chain
- Parallel chains are used to run multiple runnables in parallel.
- The final return value is a dict with the results of each value under its appropriate key.

In [14]:
system = SystemMessagePromptTemplate.from_template('Eres un profesor de {school}. Genera respuestas cortas.')

question = HumanMessagePromptTemplate.from_template('dime acerca de {topics} en {points} puntos')


messages = [system, question]
template = ChatPromptTemplate(messages)
fact_chain = template | llm | StrOutputParser()

output = fact_chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 2})
print(output)

Claro, aquí te presento dos puntos clave sobre el sistema solar:

1. **Composición del sistema solar**: El sistema solar está compuesto por 8 planetas (Mercurio, Venus, Tierra, Marte, Júpiter, Saturno, Urano y Neptuno), más luna, asteroides, cometas y otros objetos celestes.

2. **Orbita de los planetas**: Los planetas orbitan alrededor del Sol, el centro del sistema solar, siguiendo trayectorias ellipticas que pueden variar en cuanto a la distancia y velocidad.


In [16]:
question = HumanMessagePromptTemplate.from_template('Escribe un poema acerca de {topics} en {sentences} lineas')


messages = [system, question]
template = ChatPromptTemplate(messages)
poem_chain = template | llm | StrOutputParser()

output = poem_chain.invoke({'school': 'primary', 'topics': 'solar system', 'sentences': 4})
print(output)

En el cielo, una danza grande,
La Luna brilla con su luna serena y tranquila, 
Mercurio salta cerca del Sol brillante, 
Júpiter es un gigante del espacio.


In [17]:
from langchain_core.runnables import RunnableParallel

In [18]:
chain = RunnableParallel(fact = fact_chain, poem = poem_chain)

In [19]:
output = chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 2, 'sentences': 2})
print(output['fact'])
print('\n\n')
print(output['poem'])

¡Claro! Aquí te presento dos puntos sobre el Sistema Solar:

1. **El Sistema Solar consta de 8 planetas principales**: Mercurio, Venus, Tierra, Marte, Júpiter, Saturno, Urano y Neptuno.
2. **La Luna orbita alrededor de la Tierra**, siendo un satélite natural del planeta y una parte integral del Sistema Solar.



En el cosmos vasto, revoluciona la Tierra,
Con luna y estrellas, una danza sin fin.


### Chain Router
- The router chain is used to route the output of a previous runnable to the next runnable based on the output of the previous runnable.

In [20]:
prompt = """Given the user review below, classify it as either being about `Positive` or `Negative`.
            Do not respond with more than one word.

            Review: {review}
            Classification:"""

template = ChatPromptTemplate.from_template(prompt)

chain = template | llm | StrOutputParser()

review = "Thank you so much for providing such a great plateform for learning. I am really happy with the service."
# review = "I am not happy with the service. It is not good."
chain.invoke({'review': review})

'Positive'

In [21]:
positive_prompt = """
                You are expert in writing reply for positive reviews.
                You need to encourage the user to share their experience on social media.
                Review: {review}
                Answer:"""

positive_template = ChatPromptTemplate.from_template(positive_prompt)
positive_chain = positive_template | llm | StrOutputParser()

In [22]:
negative_prompt = """
                You are expert in writing reply for negative reviews.
                You need first to apologize for the inconvenience caused to the user.
                You need to encourage the user to share their concern on following Email:'udemy@kgptalkie.com'.
                Review: {review}
                Answer:"""


negative_template = ChatPromptTemplate.from_template(negative_prompt)
negative_chain = negative_template | llm | StrOutputParser()

In [23]:
def rout(info):
    if 'positive' in info['sentiment'].lower():
        return positive_chain
    else:
        return negative_chain

In [24]:
rout({'sentiment': 'negetive'})

ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template="\n                You are expert in writing reply for negative reviews.\n                You need first to apologize for the inconvenience caused to the user.\n                You need to encourage the user to share their concern on following Email:'udemy@kgptalkie.com'.\n                Review: {review}\n                Answer:"), additional_kwargs={})])
| ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')
| StrOutputParser()

In [25]:
from langchain_core.runnables import RunnableLambda

In [26]:
full_chain = {"sentiment": chain, 'review': lambda x: x['review']} | RunnableLambda(rout)

In [27]:
full_chain

{
  sentiment: ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template='Given the user review below, classify it as either being about `Positive` or `Negative`.\n            Do not respond with more than one word.\n\n            Review: {review}\n            Classification:'), additional_kwargs={})])
             | ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')
             | StrOutputParser(),
  review: RunnableLambda(lambda x: x['review'])
}
| RunnableLambda(rout)

In [28]:
# review = "Thank you so much for providing such a great plateform for learning. I am really happy with the service."
review = "I am not happy with the service. It is not good."

output = full_chain.invoke({'review': review})
print(output)

Dear valued customer,

I am writing to apologize sincerely for the inconvenience you have experienced with our service. We are truly sorry that we have fallen short of your expectations and regret that you are not satisfied with our performance.

At [Course/Platform Name], we take all feedback seriously and would like to learn more about what went wrong. Your satisfaction is our top priority, and I want to assure you that we are committed to making it right.

If you would be willing, could you please share your concerns with us in more detail? Your input will help us identify areas for improvement and ensure that we deliver a better experience for all our users in the future.

To facilitate this process, please send any additional comments or feedback regarding your experience with our service to [udemy@kgptalkie.com](mailto:udemy@kgptalkie.com). Your anonymity will be respected, and I assure you that all communications will remain confidential.

Thank you for taking the time to share 

### Make Custom Chain Runnables with RunnablePassthrough and RunnableLambda
- This is useful for formatting or when you need functionality not provided by other LangChain components, and custom functions used as Runnables are called RunnableLambdas.



In [29]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [33]:
def char_counts(text):
    return len(text)

def word_counts(text):
    return len(text.split())

prompt = ChatPromptTemplate.from_template("Explica estas entradas en 5 oraciones: {input1} y {input2}")

In [34]:
prompt

ChatPromptTemplate(input_variables=['input1', 'input2'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input1', 'input2'], input_types={}, partial_variables={}, template='Explica estas entradas en 5 oraciones: {input1} y {input2}'), additional_kwargs={})])

In [35]:
chain = prompt | llm | StrOutputParser()

output = chain.invoke({'input1': 'Earth is planet', 'input2': 'Sun is star'})

print(output)

En esta entrada, se establece la relación entre dos entidades celestes: la Tierra (planeta) y el Sol (estrella). La Tierra es un planeta que orbita alrededor del Sol. En cambio, el Sol es una estrella, cuya energía es lo que hace posible la existencia de la Tierra. Esta distinción es importante para entender la estructura y funcionamiento del sistema solar. Ambas entidades son fundamentales en nuestro universo.


In [36]:
chain = prompt | llm | StrOutputParser() | {'char_counts': RunnableLambda(char_counts), 
                                            'word_counts': RunnableLambda(word_counts), 
                                            'output': RunnablePassthrough()}

output = chain.invoke({'input1': 'Earth is planet', 'input2': 'Sun is star'})

print(output)

{'char_counts': 566, 'word_counts': 101, 'output': 'La primera entrada, "Earth es planeta", indica que la Tierra es un objeto celeste que orbita alrededor del Sol. En astronomía, un planeta es una masa celestial que orbita alrededor de otra masa celestial más grande, como el Sol. La Tierra cumple con este criterio y por eso se considera un planeta. El segundo punto dice "Sun es estrella", lo cual es cierto ya que el Sol es una masa celestial calentada principalmente por la fusión nuclear en su núcleo. En astronomía, una estrella es un objeto celeste que emite luz y calor debido a la fusión nuclear en su núcleo.'}


### Custom Chain using `@chain` decorator

In [37]:
from langchain_core.runnables import chain

In [38]:
@chain
def custom_chain(params):
    return {
        'fact': fact_chain.invoke(params),
        'poem': poem_chain.invoke(params),
    }


params = {'school': 'primary', 'topics': 'solar system', 'points': 2, 'sentences': 2}
output = custom_chain.invoke(params)
print(output['fact'])
print('\n\n')
print(output['poem'])

Aquí te presento dos puntos clave sobre el sistema solar:

1. **El Sol es la estrella central**: Es la fuente de luz y calor para todo el sistema solar.
2. **Los planetas orbitan alrededor del Sol**: Cada planeta tiene su propia órbita, trayectoria que sigue alrededor del Sol debido a su gravedad.



En el espacio, un sistema grandioso,
Sol, luna y planetas, una danza celestial, sin fin y mágico.
